#### <div style= "font-family: Cambria; font-weight:bold; letter-spacing: 0px; color:black; font-size:150%; text-align:left;padding:3.0px; background: #cceeff; border-bottom: 8px solid #004466" > TABLE OF CONTENTS<br><div>  
* [IMPORTS](#1)
* [INTRODUCTION](#2)
    * [CONFIGURATION](#2.1)
    * [CONFIGURATION PARAMETERS](#2.2)    
    * [DATASET COLUMNS](#2.3)
* [PREPROCESSING](#3)
* [MODEL TRAINING](#4)      
* [PLANNED WAY FORWARD](#5)     

<a id="1"></a>
# <div style= "font-family: Cambria; font-weight:bold; letter-spacing: 0px; color:black; font-size:120%; text-align:left;padding:3.0px; background: #cceeff; border-bottom: 8px solid #004466" > IMPORTS<br> <div> 

Credit @ravi20076
https://www.kaggle.com/code/ravi20076/playgrounds4e01-baseline-v2

In [1]:
%%time 

# Installing select libraries:-
from gc import collect;
from warnings import filterwarnings;
filterwarnings('ignore');
from IPython.display import display_html, clear_output;

! python -m pip install --no-index \
--find-links=/kaggle/input/packageinstallation \
-r /kaggle/input/packageinstallation/requirements.txt -q;
clear_output();

import xgboost as xgb, lightgbm as lgb, catboost as cb, sklearn as sk;
print(f"---> XGBoost = {xgb.__version__} | LightGBM = {lgb.__version__} | Catboost = {cb.__version__}");
print(f"---> Sklearn = {sk.__version__}\n\n")
collect();

---> XGBoost = 2.0.3 | LightGBM = 4.2.0 | Catboost = 1.2.2
---> Sklearn = 1.3.0


CPU times: total: 656 ms
Wall time: 2.87 s


In [2]:
%%time

# General library imports:-
from copy import deepcopy;
import pandas as pd;
import numpy as np;
import re;
from scipy.stats import mode, kstest, normaltest, shapiro, anderson, jarque_bera;
from collections import Counter;
from itertools import product;
from colorama import Fore, Style, init;
from warnings import filterwarnings;
filterwarnings('ignore');
import joblib;
import os;

from tqdm.notebook import tqdm;
import seaborn as sns;
import matplotlib.pyplot as plt;
from matplotlib.colors import ListedColormap as LCM;
%matplotlib inline

from pprint import pprint;
from functools import partial;

print();
collect();
clear_output();

CPU times: total: 62.5 ms
Wall time: 391 ms


In [4]:
%%time 

# Importing model and pipeline specifics:-
from category_encoders import OrdinalEncoder, OneHotEncoder;

# Pipeline specifics:-
from sklearn.preprocessing import (RobustScaler, 
                                   MinMaxScaler, 
                                   StandardScaler, 
                                   FunctionTransformer as FT,
                                   PowerTransformer,
                                  );
from sklearn.impute import SimpleImputer as SI;
from sklearn.model_selection import (RepeatedStratifiedKFold as RSKF, 
                                     StratifiedKFold as SKF,
                                     KFold, 
                                     RepeatedKFold as RKF, 
                                     cross_val_score, cross_val_predict
                                    );
from sklearn.inspection import permutation_importance;
from sklearn.feature_selection import mutual_info_classif, RFE;
from sklearn.pipeline import Pipeline, make_pipeline;
from sklearn.base import BaseEstimator, TransformerMixin;
from sklearn.compose import ColumnTransformer;

# ML Model training:-
from sklearn.metrics import roc_auc_score, brier_score_loss, make_scorer;
from xgboost import DMatrix, XGBClassifier as XGBC;
from lightgbm import log_evaluation, early_stopping, LGBMClassifier as LGBMC;
from catboost import CatBoostClassifier as CBC, Pool;
from sklearn.ensemble import (HistGradientBoostingClassifier as HGBC, 
                              RandomForestClassifier as RFC,
                              ExtraTreesClassifier as ETC,
                              GradientBoostingClassifier as GBC,
                             );
from sklearn.linear_model import LogisticRegression as LC;
from sklearn.neighbors import KNeighborsClassifier as KNNC;

# Calibration:-
from sklearn.isotonic import IsotonicRegression as ITRC;
from sklearn.calibration import CalibrationDisplay as Clb;

# Ensemble and tuning:-
import optuna;
from optuna import Trial, trial, create_study;
from optuna.pruners import HyperbandPruner;
from optuna.samplers import TPESampler, CmaEsSampler;
optuna.logging.set_verbosity = optuna.logging.CRITICAL;

clear_output();
print();
collect();


CPU times: total: 31.2 ms
Wall time: 462 ms


In [5]:
%%time 

# Setting rc parameters in seaborn for plots and graphs- 
# Reference - https://matplotlib.org/stable/tutorials/introductory/customizing.html:-
# To alter this, refer to matplotlib.rcParams.keys()

sns.set({"axes.facecolor"       : "#ffffff",
         "figure.facecolor"     : "#ffffff",
         "axes.edgecolor"       : "#000000",
         "grid.color"           : "#ffffff",
         "font.family"          : ['Cambria'],
         "axes.labelcolor"      : "#000000",
         "xtick.color"          : "#000000",
         "ytick.color"          : "#000000",
         "grid.linewidth"       : 0.75,  
         "grid.linestyle"       : "--",
         "axes.titlecolor"      : '#0099e6',
         'axes.titlesize'       : 8.5,
         'axes.labelweight'     : "bold",
         'legend.fontsize'      : 7.0,
         'legend.title_fontsize': 7.0,
         'font.size'            : 7.5,
         'xtick.labelsize'      : 7.5,
         'ytick.labelsize'      : 7.5,        
        });

# Color printing    
def PrintColor(text:str, color = Fore.BLUE, style = Style.BRIGHT):
    "Prints color outputs using colorama using a text F-string";
    print(style + color + text + Style.RESET_ALL); 

# Making sklearn pipeline outputs as dataframe:-
from sklearn import set_config; 
set_config(transform_output = "pandas");
pd.set_option('display.max_columns', 50);
pd.set_option('display.max_rows', 50);

print();
collect();



CPU times: total: 31.2 ms
Wall time: 63 ms


<a id="2"></a>
# <div style= "font-family: Cambria; font-weight:bold; letter-spacing: 0px; color:black; font-size:120%; text-align:left;padding:3.0px; background: #cceeff; border-bottom: 8px solid #004466" > INTRODUCTION<br><div> 

| Version<br>Number | Version Details | CV score| Single/ Ensemble|Public LB Score|
| :-: | --- | :-: | :-: |:-:|
| **V1** |* Public notebook features <br> * 10x1 ML models training <br> * Optuna ensemble <br> * Blended with public work|0.89284|Ensemble <br> Optuna|0.89316|
| **V2** |* Same as V1 <br> * Included the post-processing trick|0.89284|Ensemble <br> Optuna||

<a id="2.1"></a>
## <div style= "font-family: Cambria; font-weight:bold; letter-spacing: 0px; color:#ffffff; font-size:120%; text-align:left;padding:3.0px; background: #0059b3; border-bottom: 8px solid #e6e6e6" > CONFIGURATION<br><div> 

In [16]:
%%time

# Configuration class:-
class CFG:
    """
    Configuration class for parameters and CV strategy for tuning and training
    Some parameters may be unused here as this is a general configuration class
    """;
    
    # Data preparation:-   
    version_nb         = 2;
    test_req           = "N";
    test_sample_frac   = 0.025;
    gpu_switch         = "OFF"; 
    state              = 42;
    target             = 'Exited';
    episode            = 1;
    path               = f"\data";
    orig_path          = f"\data\Churn_Modelling.csv";
    data_path          = f"\data";
    
    dtl_preproc_req    = "Y";
    adv_cv_req         = "N";
    ftre_plots_req     = 'N';
    ftre_imp_req       = "N";
    
    # Data transforms and scaling:-    
    conjoin_orig_data  = "N";
    drop_nulls         = "Y";
    sec_ftre_req       = "Y";
    scale_req          = "N";
    # NOTE---Keep a value here even if scale_req = N, this is used for linear models:-
    scl_method         = "Z"; 
    enc_method         = 'Label';
    OH_cols            = [];
    
    # Model Training:- 
    baseline_req       = "N";
    pstprcs_oof        = "N";
    pstprcs_train      = "N";
    pstprcs_test       = "N";
    ML                 = "Y";
    
    pseudo_lbl_req     = "N";
    pseudolbl_up       = 0.975;
    pseudolbl_low      = 0.01;
    
    use_orig_allfolds  = "N";
    n_splits           = 3 if test_req == "Y" else 10;
    n_repeats          = 1 ;
    nbrnd_erly_stp     = 100;
    mdlcv_mthd         = 'RSKF';
    
    # Ensemble:-    
    ensemble_req       = "Y";
    hill_climb_req     = "N";
    optuna_req         = "Y";
    LAD_req            = "N";
    enscv_mthd         = "RSKF";
    metric_obj         = 'maximize';
    ntrials            = 10 if test_req == "Y" else 150;
    
    # Global variables for plotting:-
    grid_specs = {'visible': True, 'which': 'both', 'linestyle': '--', 
                           'color': 'lightgrey', 'linewidth': 0.75};
    title_specs = {'fontsize': 9, 'fontweight': 'bold', 'color': 'tab:blue'};

print();
PrintColor(f"--> Configuration done!\n");
collect();

SyntaxError: unterminated string literal (detected at line 16) (<unknown>, line 16)

In [12]:
%%time 

# Defining functions to be used throughout the code for common tasks:-

# Scaler to be used for continuous columns:- 
all_scalers = {'Robust': RobustScaler(), 
               'Z': StandardScaler(), 
               'MinMax': MinMaxScaler(),
               "PT" : PowerTransformer(),
              };
scaler      = all_scalers.get(CFG.scl_method);

# Making a directory to store the model objects:-
if os.path.exists("/kaggle/working/MLModels") == True:
    PrintColor(f"\n---> MLModels already exists\n", color = Fore.RED);
else:
    os.mkdir("MLModels");
    PrintColor(f"\n---> Made directory MLModels to store models for inference\n", color = Fore.RED)


collect();
print();


FileExistsError: [WinError 183] 파일이 이미 있으므로 만들 수 없습니다: 'MLModels'

<a id="2.2"></a>
## <div style= "font-family: Cambria; font-weight:bold; letter-spacing: 0px; color:#ffffff; font-size:120%; text-align:left;padding:3.0px; background: #0059b3; border-bottom: 8px solid #e6e6e6" > CONFIGURATION PARAMETERS<br><div> 


| Parameter         | Description                                             | Possible value choices|
| ---               | ---                                                     | :-:                   |
|  version_nb       | Version Number                                          | integer               |
|  test_req         | Are we testing syntax here?                             | Y/N                   |  
|  test_sample_frac | Sample size for syntax test                             | float(0-1)/ int       |     
|  gpu_switch       | GPU switch                                              | ON/OFF                |
|  state            | Random state for most purposes                          | integer               |
|  target           | Target column name                                      | yield                 |
|  episode          | Episode Number                                          | integer               |
|  path             | Path for input data files                               |                       |
|  orig_path        | Path for input original data files                      |                       |
|  data_path        | Path for FE data files                                  |                       |   
|  dtl_preproc_req  | Proprocessing required                                  | Y/N                   |    
|  adv_cv_req       | Adversarial CV required                                 | Y/N                   |
|  ftre_plots_req   | Feature plots required                                  | Y/N                   |
|  ftre_imp_req     | Feature importance required                             | Y/N                   |
|  conjoin_orig_data| Conjoin original data                                   | Y/N                   |
|  drop_nulls       | Drop original data nulls                                | Y/N                   |    
|  sec_ftre_req     | Secondary features required                             | Y/N                   |
|  scale_req        | Scaling required                                        | Y/N                   |
|  scl_method       | Scaling method                                          | Z/ Robust/ MinMax     |
|  enc_method       | Encoding method                                         |-                      |
|  OH_cols          | Onehot columns                                          |list                   |
|  tgt_mapper       | Target categories                                       |dict                   |   
|  baseline_req     | Baseline model required                                 | Y/N                   |
|  pstprcs_oof      | Post-process OOF after model training                   | Y/N                   |
|  pstprcs_train    | Post-process OOF during model training for dev-set      | Y/N                   |
|  pstprcs_test     | Post-process test after training                        | Y/N                   |
|  ML               | Machine Learning Models                                 | Y/N                   |
|  use_orig_all_folds| Use original data in all folds                         | Y/N                   |
|  n_splits         | Number of CV splits                                     | integer               |
|  n_repeats        | Number of CV repeats                                    | integer               |
|  nbrnd_erly_stp   | Number of early stopping rounds                         | integer               |
|  mdl_cv_mthd      | Model CV method name                                    | RKF/ RSKF/ SKF/ KFold |

<a id="2.3"></a>
## <div style= "font-family: Cambria; font-weight:bold; letter-spacing: 0px; color:#ffffff; font-size:120%; text-align:left;padding:3.0px; background: #0059b3; border-bottom: 8px solid #e6e6e6" > DATASET AND COMPETITION DETAILS<br><div>
    
**Data columns**<br>
This is available in the original data description as below<br>
https://www.kaggle.com/datasets/shubhammeshram579/bank-customer-churn-prediction <br>
Feature engineering dataset is taken from the public notebook <br>
https://www.kaggle.com/code/arunklenin/ps4e1-advanced-feature-engineering-ensemble <br>
<br>**Competition details and notebook objectives**<br>
1. This is a binary-class classification challenge to predict bank customer churn. **ROC-AUC score** is the metric for the challenge<br>
2. In this starter notebook, we start the assignment with a detailed EDA, feature plots, interaction effects, adversarial CV analysis and develop starter models to initiate the challenge. We will also incorporate other opinions and approaches as we move along the challenge.<br>
<br>
**Model strategy** <br>
We start off with simple tree based ML models and an Optuna ensemble to create sample inputs for the submission. <br>


<a id="3"></a>
# <div style= "font-family: Cambria; font-weight:bold; letter-spacing: 0px; color:black; font-size:120%; text-align:left;padding:3.0px; background: #cceeff; border-bottom: 8px solid #004466" > PREPROCESSING<br><div> 

In [13]:
%time 

class Preprocessor():
    """
    This class aims to do the below-
    1. Read the datasets
    2. In this case, process the original data
    3. Check information and description
    4. Check unique values and nulls
    5. Collate starting features 
    6. Conjoin train-original data if requested based on Adversarial CV results
    """;
    
    def __init__(self):
        self.train             = pd.read_csv(os.path.join(CFG.data_path,"train.csv"), index_col = ["Unnamed: 0"]);
        self.test              = pd.read_csv(os.path.join(CFG.data_path ,"test.csv"), index_col = ["Unnamed: 0"]);
        self.target            = CFG.target ;
        self.original          = pd.read_csv(CFG.orig_path);
        self.conjoin_orig_data = CFG.conjoin_orig_data;
        self.dtl_preproc_req   = CFG.dtl_preproc_req;
        self.test_req          = CFG.test_req;
        
        self.sub_fl   = pd.read_csv(os.path.join(CFG.path, "sample_submission.csv"));
        PrintColor(f"Data shapes - train-test-original = {self.train.shape} {self.test.shape} {self.original.shape}");
        
        for tbl in [self.train, self.test]:
            tbl.columns = tbl.columns.str.replace(r"\(|\)|\s+","", regex = True);
            
        PrintColor(f"\nTrain set head", color = Fore.GREEN);
        display(self.train.head(5).style.format(precision = 3));
        PrintColor(f"\nTest set head", color = Fore.GREEN);
        display(self.test.head(5).style.format(precision = 3));
                         
    def _AddSourceCol(self):
        self.train.index.name = "id";
        self.test.index.name = "id";
        
        self.train['Source'] = "Competition";
        self.test['Source']  = "Competition";
        
        self.strt_ftre = self.test.columns;
        return self;
          
    def _CollateInfoDesc(self):
        if self.dtl_preproc_req == "Y":
            PrintColor(f"\n{'-'*20} Information and description {'-'*20}\n", color = Fore.MAGENTA);

            # Creating dataset information and description:
            for lbl, df in {'Train': self.train, 'Test': self.test}.items():
                PrintColor(f"\n{lbl} description\n");
                display(df.describe(percentiles= [0.05, 0.25, 0.50, 0.75, 0.9, 0.95, 0.99]).\
                        transpose().\
                        drop(columns = ['count'], errors = 'ignore').\
                        drop([CFG.target], axis=0, errors = 'ignore').\
                        style.format(formatter = '{:,.2f}').\
                        background_gradient(cmap = 'Blues')
                       );

                PrintColor(f"\n{lbl} information\n");
                display(df.info());
                collect();
        return self;
    
    def _CollateUnqNull(self):
        
        if self.dtl_preproc_req == "Y":
            # Dislaying the unique values across train-test-original:-
            PrintColor(f"\nUnique and null values\n");
            _ = pd.concat([self.train[self.strt_ftre].nunique(), 
                           self.test[self.strt_ftre].nunique(), 
                           self.train[self.strt_ftre].isna().sum(axis=0),
                           self.test[self.strt_ftre].isna().sum(axis=0),
                          ], 
                          axis=1);
            _.columns = ['Train_Nunq', 'Test_Nunq', 'Train_Nulls', 'Test_Nulls',];

            display(_.T.style.background_gradient(cmap = 'Blues', axis=1).\
                    format(formatter = '{:,.0f}')
                   );
            
        return self;
       
    def DoPreprocessing(self):
        self._AddSourceCol();
        self._CollateInfoDesc();
        self._CollateUnqNull();
        
        return self; 
        
    def ConjoinTrainOrig(self):
        if self.conjoin_orig_data == "Y":
            PrintColor(f"Train shape before conjoining with original = {self.train.shape}");
            train = pd.concat([self.train, self.original], axis=0, ignore_index = True);
            PrintColor(f"Train shape after conjoining with original= {train.shape}");
            
            train = train.drop_duplicates();
            PrintColor(f"Train shape after de-duping = {train.shape}");
            
            train.index = range(len(train));
            train.index.name = 'id';
        
        else:
            PrintColor(f"We are using the competition training data only");
            train = self.train;
        return train;
          
collect();
print();

CPU times: total: 0 ns
Wall time: 0 ns



In [14]:
%%time 

pp = Preprocessor();
pp.DoPreprocessing();

print();
train, test, strt_ftre = pp.ConjoinTrainOrig(), pp.test.copy(deep = True), deepcopy(pp.strt_ftre);

print();
collect();

FileNotFoundError: [Errno 2] No such file or directory: '\\data\\train.csv'

In [ ]:
%%time 

# Calculating skewness across columns:-
skew_df = pd.DataFrame(index = pp.train.drop(columns = ["Source", CFG.target], errors = "ignore").columns);
for col, df in {"Train": pp.train.drop(columns = ["Source", CFG.target], errors = "ignore"), 
                "Test": pp.test.drop(columns = ["Source"], errors = "ignore")}.items():   
    skew_df = \
    pd.concat([skew_df, 
               df.drop(columns = [CFG.target, "Source", "id"], errors = "ignore").skew()],
               axis=1).rename({0: col}, axis=1);

PrintColor(f"\nSkewness across independent features\n");
display(skew_df.transpose().style.format(precision = 2).background_gradient("Pastel2"));

collect();
print();

In [ ]:
%%time

PrintColor(f"\n{'='* 20} Data transformation {'='* 20} \n");

ytrain = train[CFG.target];
Xtrain = train.drop(columns = [CFG.target]);
Xtest  = test.copy(deep = True);

PrintColor(f"\n---> Train data\n");
display(Xtrain.head(5).style.format(precision = 2));
PrintColor(f"\n---> Test data\n");
display(Xtest.head(5).style.format(precision = 2));

print();
PrintColor(f"\n---> Dropping duplicates in the train data");
_ = pd.concat([Xtrain, ytrain], axis=1).drop_duplicates(subset = Xtrain.columns[0:-1].to_list());
_.index = range(len(_));
Xtrain, ytrain = _.drop(columns = [CFG.target]), _[CFG.target];
del _;
PrintColor(f"---> Train-test shape after duplicate removal = {Xtrain.shape} {Xtest.shape}");

print();
collect();


In [ ]:
%%time 

# Sampling the data for testing purposes:-
if CFG.test_req == "Y":
    _ = pd.concat([Xtrain, ytrain], axis=1).groupby([CFG.target]).sample(frac = CFG.test_sample_frac);
    Xtrain = _.drop(columns = [CFG.target]);
    ytrain = _[CFG.target];
    Xtrain.index = range(len(Xtrain));
    ytrain.index = Xtrain.index;
    del _;
    
    PrintColor(f"\n---> Train size after sampling for syntax check = {Xtrain.shape} {ytrain.shape}");
    
collect();
print();

<a id="7"></a>
# <div style= "font-family: Cambria; font-weight:bold; letter-spacing: 0px; color:black; font-size:120%; text-align:left;padding:3.0px; background: #cceeff; border-bottom: 8px solid #004466" > MODEL TRAINING <br><div> 
   

In [ ]:
%%time 

class OptunaEnsembler:
    """
    This is the Optuna ensemble class-
    Source- https://www.kaggle.com/code/arunklenin/ps3e26-cirrhosis-survial-prediction-multiclass
    """;
    
    def __init__(self):
        self.study        = None;
        self.weights      = None;
        self.random_state = CFG.state;
        self.n_trials     = CFG.ntrials;
        self.direction    = CFG.metric_obj;
        
    def ScoreMetric(self, ytrue, ypred):
        """
        This is the metric function for the competition
        """;
        return roc_auc_score(ytrue, ypred);

    def _objective(self, trial, y_true, y_preds):
        """
        This method defines the objective function for the ensemble
        """;
        
        if isinstance(y_preds, pd.DataFrame) or isinstance(y_preds, np.ndarray):
            weights = [trial.suggest_float(f"weight{n}", 0, 1) for n in range(y_preds.shape[-1])];
            axis = 1;
        elif isinstance(y_preds, list):
            weights = [trial.suggest_float(f"weight{n}", 0, 1) for n in range(len(y_preds))];
            axis = 0;

        # Calculating the weighted prediction:-
        weighted_pred  = np.average(np.array(y_preds), axis = axis, weights = weights);
        score          = self.ScoreMetric(y_true, weighted_pred);
        return score;

    def fit(self, y_true, y_preds):
        "This method fits the Optuna objective on the fold level data";
        
        optuna.logging.set_verbosity = optuna.logging.ERROR;
        self.study = \
        optuna.create_study(sampler    = TPESampler(seed = self.random_state), 
                            pruner     = HyperbandPruner(),
                            study_name = "Ensemble", 
                            direction  = self.direction,
                           );
        
        obj = partial(self._objective, y_true = y_true, y_preds = y_preds);
        self.study.optimize(obj, n_trials = self.n_trials);
               
        if isinstance(y_preds, list):
            self.weights = [self.study.best_params[f"weight{n}"] for n in range(len(y_preds))];
        else:
            self.weights = [self.study.best_params[f"weight{n}"] for n in range(y_preds.shape[-1])];
        clear_output();

    def predict(self, y_preds):
        "This method predicts using the fitted Optuna objective";
        
        assert self.weights is not None, 'OptunaWeights error, must be fitted before predict';
        
        if isinstance(y_preds, list):
            weighted_pred = np.average(np.array(y_preds), axis=0, weights = self.weights);
        else:
            weighted_pred = np.average(np.array(y_preds), axis=1, weights = self.weights);
        return weighted_pred;

    def fit_predict(self, y_true, y_preds):
        """
        This method fits the Optuna objective on the fold data, then predicts the test set
        """;
        self.fit(y_true, y_preds);
        return self.predict(y_preds);
    
    def weights(self):
        return self.weights;
    
print();
collect();

In [ ]:
%%time 

class MdlDeveloper(CFG):
    """
    This class implements the training pipeline elements-
    1. Initializes the Model predictions
    2. Trains and infers models
    3. Returns the OOF and model test set predictions
    """;
    
    def __init__(self, Xtrain, ytrain, Xtest, sel_cols, cat_cols, **kwarg):
        """
        In this method, we initialize the below-
        1. Train-test data, selected columns
        2. Metric, custom scorer, model and cv object
        3. Output tables for score and predictions
        """;
        
        self.Xtrain      = Xtrain;
        self.ytrain      = ytrain;
        self.y_grp       = ytrain;
        self.Xtest       = Xtest;
        self.sel_cols    = sel_cols;
        self.cat_cols    = cat_cols;
     
        self._DefineModels();
        self.cv          = self.all_cv[self.mdlcv_mthd];
        self.methods     = list(self.Mdl_Master.keys());
        self.OOF_Preds   = pd.DataFrame();
        self.Mdl_Preds   = pd.DataFrame();
        self.Scores      = pd.DataFrame(columns = self.methods + ["Ensemble"], 
                                        index = range(self.n_splits * self.n_repeats)
                                       ); 
        self.TrainScores = pd.DataFrame(columns = self.methods, 
                                        index = range(self.n_splits * self.n_repeats)
                                       ); 
        self.mdlscorer   = make_scorer(self.ScoreMetric, 
                                       greater_is_better = True,
                                       needs_proba       = True,
                                       needs_threshold   = False,
                                      );  
        
        PrintColor(f"\n---> Selected model options-");
        try:
            with np.printoptions(linewidth = 150):
                pprint(np.array(self.methods), depth = 1, width = 100, indent = 5);
        except:
            pprint(self.methods, depth = 1, width = 100, indent = 5);
              
    def _DefineModels(self):
        """
        This method initiliazes models for the analysis
        It also initializes the CV methods and class-weights that could be tuned going ahead.
        """;
        
        # Commonly used CV strategies for later usage:-
        self.all_cv = \
        {'KF'  : KFold(n_splits= CFG.n_splits, shuffle = True, random_state= CFG.state),
         'RKF' : RKF(n_splits= CFG.n_splits, n_repeats = CFG.n_repeats, random_state= CFG.state),
         'RSKF': RSKF(n_splits= CFG.n_splits, n_repeats = CFG.n_repeats, random_state= CFG.state),
         'SKF' : SKF(n_splits= CFG.n_splits, shuffle = True, random_state= CFG.state)
        };
           
        self.Mdl_Master = \
        {                      
         'XGB1C': XGBC(**{'tree_method'           : "gpu_hist" if CFG.gpu_switch == "ON" else "hist",
                          'objective'             : 'binary:logistic',
                          'eval_metric'           : "auc",
                          'random_state'          : self.state,
                          'colsample_bytree'      : 0.25,
                          'learning_rate'         : 0.07,
                          'max_depth'             : 8,
                          'n_estimators'          : 1100,                         
                          'reg_alpha'             : 0.09,
                          'reg_lambda'            : 0.70,
                          'min_child_weight'      : 22,
                          'early_stopping_rounds' : CFG.nbrnd_erly_stp,
                          'verbosity'             : 0,
                         }
                      ),
            
         'XGB2C': XGBC(**{'tree_method'           : "gpu_hist" if CFG.gpu_switch == "ON" else "hist",
                          'objective'             : 'binary:logistic',
                          'eval_metric'           : "auc",
                          'random_state'          : self.state,
                          'colsample_bytree'      : 0.40,
                          'learning_rate'         : 0.06,
                          'max_depth'             : 9,
                          'n_estimators'          : 2500,                         
                          'reg_alpha'             : 0.12,
                          'reg_lambda'            : 0.8,
                          'min_child_weight'      : 15,
                          'early_stopping_rounds' : CFG.nbrnd_erly_stp,
                          'verbosity'             : 0,
                         }
                      ),

         'XGB3C': XGBC(**{'tree_method'           : "gpu_hist" if CFG.gpu_switch == "ON" else "hist",
                          'objective'             : 'binary:logistic',
                          'eval_metric'           : "auc",
                          'random_state'          : self.state,
                          'colsample_bytree'      : 0.5,
                          'learning_rate'         : 0.055,
                          'max_depth'             : 9,
                          'n_estimators'          : 3000,                         
                          'reg_alpha'             : 0.2,
                          'reg_lambda'            : 0.6,
                          'min_child_weight'      : 25,
                          'early_stopping_rounds' : CFG.nbrnd_erly_stp,
                          'verbosity'             : 0,
                         }
                      ),
            
         'XGB4C': XGBC(**{'tree_method'           : "gpu_hist" if CFG.gpu_switch == "ON" else "hist",
                          'objective'             : 'binary:logistic',
                          'eval_metric'           : "auc",
                          'random_state'          : self.state,
                          'colsample_bytree'      : 0.80,
                          'learning_rate'         : 0.082,
                          'max_depth'             : 7,
                          'n_estimators'          : 2000,                         
                          'reg_alpha'             : 0.005,
                          'reg_lambda'            : 0.95,
                          'min_child_weight'      : 26,
                          'early_stopping_rounds' : CFG.nbrnd_erly_stp,
                          'verbosity'             : 0,
                         }
                      ),
              
         'LGBM1C':LGBMC(**{'device'              : "gpu" if CFG.gpu_switch == "ON" else "cpu",
                           'objective'           : 'binary',
                           'boosting_type'       : 'gbdt',
                           'metric'              : "auc",
                           'random_state'        : self.state,
                           'colsample_bytree'    : 0.56,
                           'subsample'           : 0.35,
                           'learning_rate'       : 0.05,
                           'max_depth'           : 8,
                           'n_estimators'        : 3000,
                           'num_leaves'          : 140,
                           'reg_alpha'           : 0.14,
                           'reg_lambda'          : 0.85,
                           'verbosity'           : -1, 
                          }
                       ),
            
         'LGBM2C':LGBMC(**{'device'              : "gpu" if CFG.gpu_switch == "ON" else "cpu",
                           'objective'           : 'binary',
                           'boosting_type'       : 'gbdt',
                           'data_sample_strategy': "goss",
                           'metric'              : "auc",
                           'random_state'        : self.state,
                           'colsample_bytree'    : 0.20,
                           'subsample'           : 0.25,
                           'learning_rate'       : 0.0725,
                           'max_depth'           : 9,
                           'n_estimators'        : 3000,
                           'num_leaves'          : 120, 
                           'reg_alpha'           : 0.15,
                           'reg_lambda'          : 0.90,
                           'verbosity'           : -1, 
                          }
                       ),
            
         'LGBM3C':LGBMC(**{'device'              : "gpu" if CFG.gpu_switch == "ON" else "cpu",
                           'objective'           : 'binary',
                           'boosting_type'       : 'gbdt',
                           'metric'              : "auc",
                           'random_state'        : self.state,
                           'colsample_bytree'    : 0.45,
                           'subsample'           : 0.45,
                           'learning_rate'       : 0.06,
                           'max_depth'           : 6,
                           'n_estimators'        : 3000,
                           'num_leaves'          : 125, 
                           'reg_alpha'           : 0.05,
                           'reg_lambda'          : 0.95,
                           'verbosity'           : -1,
                          }
                       ), 
            
         'LGBM4C':LGBMC(**{'device'              : "gpu" if CFG.gpu_switch == "ON" else "cpu",
                           'objective'           : 'binary',
                           'boosting_type'       : 'gbdt',
                           'metric'              : "auc",
                           'random_state'        : self.state,
                           'colsample_bytree'    : 0.55,
                           'subsample'           : 0.55,
                           'learning_rate'       : 0.085,
                           'max_depth'           : 9,
                           'n_estimators'        : 3000,
                           'num_leaves'          : 105, 
                           'reg_alpha'           : 0.08,
                           'reg_lambda'          : 0.995,
                           'verbosity'           : -1,
                          }
                       ),
            
        "CB1C" :  CBC(**{'task_type'           : "GPU" if CFG.gpu_switch == "ON" else "CPU",
                         'objective'           : 'Logloss',
                         'eval_metric'         : "AUC",
                         'bagging_temperature' : 0.1,
                         'colsample_bylevel'   : 0.88,
                         'iterations'          : 3000,
                         'learning_rate'       : 0.065,
                         'od_wait'             : 12,
                         'max_depth'           : 7,
                         'l2_leaf_reg'         : 1.75,
                         'min_data_in_leaf'    : 25,
                         'random_strength'     : 0.1, 
                         'max_bin'             : 100,
                         'verbose'             : 0,
                         'use_best_model'      : True,
                        }
                     ),
            
        "CB2C" :  CBC(**{'task_type'           : "GPU" if CFG.gpu_switch == "ON" else "CPU",
                         'objective'           : 'Logloss',
                         'eval_metric'         : "AUC",
                         'bagging_temperature' : 0.5,
                         'colsample_bylevel'   : 0.50,
                         'iterations'          : 2500,
                         'learning_rate'       : 0.08,
                         'od_wait'             : 24,
                         'max_depth'           : 8,
                         'l2_leaf_reg'         : 1.235,
                         'min_data_in_leaf'    : 25,
                         'random_strength'     : 0.35, 
                         'max_bin'             : 160,
                         'verbose'             : 0,
                         'use_best_model'      : True,
                        }
                     ),
        
        "GBC" : GBC(loss                = 'log_loss',
                    learning_rate       = 0.065,
                    n_estimators        = 850,
                    subsample           = 0.6,
                    min_samples_leaf    = 30,
                    max_depth           = 8,
                    random_state        = self.state,
                    verbose             = 0,
                    validation_fraction = 0.1,
                    n_iter_no_change    = 50,
                   ),
            
        "HGBC": HGBC(loss                = 'log_loss',
                     learning_rate       = 0.065,
                     max_iter            = 450,
                     max_depth           = 8,
                     min_samples_leaf    = 35,
                     l2_regularization   = 0.85,
                     early_stopping      = 'auto',
                     scoring             = 'loss',
                     validation_fraction = 0.1,
                     n_iter_no_change    = 50,
                     random_state        = self.state,
                    ),
        };
        return self;
    
    def ScoreMetric(self, ytrue, ypred):
        """
        This is the metric function for the competition scoring
        """;
        return roc_auc_score(ytrue, ypred);
    
    def ClbMetric(self, ytrue, ypred):
        """
        This is the calibration metric
        """;
        return brier_score_loss(ytrue, ypred);
        
    def PostProcessPred(self, ypred):
        """
        This is an optional post-processing method
        """;
        return np.clip(ypred,a_min = 0,a_max = 1);
    
    def CalcSmplWgt(self):
        """
        This method calculates sample weights for the training process
        """;
        _ = self.Xtrain[["CustomerId"]];
        myser = _.groupby(['CustomerId']).size();
        myser.name = "Size";
        self.sample_weight = _.merge(myser, how = "left",  left_on = "CustomerId", right_index = True)["Size"];
        return self;
        
    def TrainMdl(self, test_preds_req: str = "Y", save_models = "N"):
        """
        This method trains and infers from the model suite and returns the predictions and scores
        It optionally predicts the test set too, if desired by the user
        """;

        # Initializing I-O:- 
        X,y, Xt    = self.Xtrain[self.sel_cols], self.ytrain.copy(deep = True), self.Xtest[self.sel_cols];
        cols_drop  = ['Source', "id"];
        ens        = OptunaEnsembler();
        
        self.FtreImp = pd.DataFrame(columns = self.methods, 
                                    index   = [c for c in self.sel_cols if c not in cols_drop]
                                   ).fillna(0);
        
        self.CalcSmplWgt();
        
        if self.test_req == "Y":
            PrintColor(f"\n---> Sample weights\n", color = Fore.RED);
            display(self.sample_weight.head(10));
            print();
        
        # Making CV folds:-        
        for fold_nb, (train_idx, dev_idx) in tqdm(enumerate(self.cv.split(X, self.y_grp))):
            Xtr  = X.iloc[train_idx].drop(columns = cols_drop, errors = 'ignore');  
            Xdev = X.iloc[dev_idx].query("Source == 'Competition'").drop(columns = cols_drop, errors = 'ignore');    
            ytr  = y.loc[y.index.isin(Xtr.index)];
            ydev = y.loc[y.index.isin(Xdev.index)];
                   
            # Initializing the OOF and test set predictions:-            
            oof_preds = pd.DataFrame(columns = self.methods, index = Xdev.index);
            mdl_preds = pd.DataFrame(columns = self.methods, index = Xt.index);
            
            PrintColor(f"\n{'=' * 5} FOLD {fold_nb + 1} {'=' * 5}\n");
            # Initializing models across methods:-
            for method in tqdm(self.methods):
                model = Pipeline(steps = [("M", self.Mdl_Master.get(method))]); 

                # Fitting the model:-          
                if "CB" in method:    
                    model.fit(Xtr, ytr, 
                              M__eval_set = [(Xdev, ydev)], 
                              M__verbose = 0,
                              M__early_stopping_rounds = CFG.nbrnd_erly_stp,
                              M__sample_weight = self.sample_weight.loc[train_idx],
                             ); 

                elif "LGBM" in method:
                    model.fit(Xtr, ytr, 
                              M__eval_set = [(Xdev, ydev)], 
                              M__callbacks = [log_evaluation(0), 
                                              early_stopping(stopping_rounds = CFG.nbrnd_erly_stp, 
                                                             verbose = False,),
                                             ],
                              M__sample_weight = self.sample_weight.loc[train_idx],
                             ); 

                elif "XGB" in method:
                     model.fit(Xtr, ytr, 
                               M__eval_set = [(Xdev, ydev)], 
                               M__verbose  = 0,
                               M__sample_weight = self.sample_weight.loc[train_idx],
                              );            

                else: 
                    try:
                        model.fit(Xtr, ytr,M__sample_weight = self.sample_weight.loc[train_idx],);
                    except:
                        model.fit(Xtr, ytr);
                        
                # Saving the model objects:-
                if save_models == "Y":
                    joblib.dump(model, f"/kaggle/working/MLModels/{method}_Fold{fold_nb}.model");
                    
                # Collating feature importance:-
                try: 
                    self.FtreImp[method] += model["M"].feature_importances_;
                except: 
                    pass;
                    
                # Collecting predictions and scores and post-processing OOF based on model method:-
                dev_preds    = model.predict_proba(Xdev)[:,1];
                train_preds  = model.predict_proba(Xtr)[:,1];
                tr_score     = self.ScoreMetric(ytr.values.flatten(),train_preds);
                score        = self.ScoreMetric(ydev.values.flatten(),dev_preds);
                         
                PrintColor(f"OOF = {score:.5f} | Train = {tr_score:.5f}| {method}",color = Fore.CYAN);
                oof_preds[method] = dev_preds;

                # Integrating the predictions and scores:-               
                self.Scores.at[fold_nb, method]      = np.round(score, decimals= 6);
                self.TrainScores.at[fold_nb, method] = np.round(tr_score, decimals= 6);
            
                if test_preds_req == "Y": 
                    mdl_preds[method] = \
                    self.PostProcessPred(model.predict_proba(Xt.drop(columns = cols_drop, errors = "ignore")));
                
            try:
                del dev_preds, train_preds, tr_score, score;
            except:
                pass;
                
            # Ensembling the predictions:-
            oof_preds["Ensemble"]  = ens.fit_predict(ydev, oof_preds[self.methods]);
            score                  = self.ScoreMetric(ydev, oof_preds["Ensemble"].values);
            self.OOF_Preds         = pd.concat([self.OOF_Preds, oof_preds], axis = 0, ignore_index = False);
            self.Scores.at[fold_nb, "Ensemble"] = np.round(score,6);
            
            if test_preds_req == "Y": 
                mdl_preds["Ensemble"] = ens.predict(mdl_preds[self.methods]);
                self.Mdl_Preds        = pd.concat([self.Mdl_Preds, mdl_preds], axis = 0, ignore_index = False);
                
        # Averaging the predictions after all folds:-       
        self.OOF_Preds = self.OOF_Preds.groupby(level = 0).mean();
        if test_preds_req == "Y": 
            self.Mdl_Preds = self.Mdl_Preds[self.methods + ["Ensemble"]].groupby(level=0).mean();
            
        return self.OOF_Preds, self.Mdl_Preds, self.Scores, self.TrainScores;
    
    def DisplayAdjTbl(self, *args):
        """
        This function displays pandas tables in an adjacent manner, sourced from the below link-
        https://stackoverflow.com/questions/38783027/jupyter-notebook-display-two-pandas-tables-side-by-side
        """;

        html_str = '';
        for df in args:
            html_str += df.to_html();
        display_html(html_str.replace('table','table style="display:inline"'),raw=True);
         
    def DisplayScores(self):
        "This method displays the scores and their means";
        
        PrintColor(f"\n---> OOF score across all methods and folds\n",color = Fore.LIGHTMAGENTA_EX);
        display(self.Scores.style.format(precision = 5).\
                background_gradient(cmap = "Pastel2", subset = self.methods).\
                background_gradient(cmap = "mako", subset = ["Ensemble"]).\
                set_caption(f"\nOOF scores across methods and folds\n")
               );
        
        PrintColor(f"\n---> Mean OOF score across all methods and folds\n");
        display(self.Scores.mean().to_frame().transpose().\
                style.\
                format(precision = 5).\
                background_gradient(cmap = "gist_earth", subset = self.methods + ["Ensemble"], axis=1).\
                set_caption(f"\nMean OOF scores across methods and folds\n")
               );
        
        PrintColor(f"\n---> Train score across all methods and folds\n",color = Fore.LIGHTMAGENTA_EX);
        display(self.TrainScores.style.format(precision = 5).\
                background_gradient(cmap = "icefire", subset = self.methods).\
                set_caption(f"\nTrain scores across methods and folds\n")
               ); 
        
        PrintColor(f"\n---> Mean Train score across all methods and folds\n");
        display(self.TrainScores.mean().to_frame().transpose().\
                style.\
                format(precision = 5).\
                background_gradient(cmap = "gist_earth", subset = self.methods, axis=1).\
                set_caption(f"\nMean Train scores across methods and folds\n")
               );
      
    def MakePseudoLbl(self, up_cutoff: float, low_cutoff: float, **kwarg):
        """
        This method makes pseudo-labels using confident test set predictions to add to the training data
        """;
        
        # Locating confident test-set predictions:-        
        df = \
        self.Mdl_Preds.loc[(self.Mdl_Preds.Ensemble >= up_cutoff) | (self.Mdl_Preds.Ensemble <= low_cutoff), 
                           "Ensemble"
                          ];
        PrintColor(f"---> Pseudo Label additions from test set = {df.shape[0]:,.0f}", color = Fore.RED);
        df = df.astype(np.uint8);
        
        #  Integrating new Xtrain and ytrain based on pseudo-labels:- 
        new_ytrain       = pd.concat([self.ytrain, df], axis=0, ignore_index = True);
        new_ytrain.index = range(len(new_ytrain));
        new_Xtrain       = pd.concat([self.Xtrain, self.Xtest.loc[df.index]], axis=0, ignore_index = True);
        new_Xtrain.index = range(len(new_Xtrain));
        
        #  Verifying the additions:-
        PrintColor(f"---> Revised train set shapes after pseudo labels = {new_Xtrain.shape} {new_ytrain.shape}");
        return new_Xtrain, new_ytrain;
    
    def MakeMLPlots(self):
        """
        This method makes plots for the ML models, including feature importance and calibration curves
        """;
        
        fig, axes = plt.subplots(len(self.methods), 2, figsize = (35, len(self.methods) * 10),
                                 gridspec_kw = {'hspace': 0.6, 'wspace': 0.2}, 
                                 width_ratios= [0.75, 0.25],
                                );
    
        for i, col in enumerate(self.methods):
            try: 
                ax = axes[i,0];
            except: 
                ax = axes[0];
                
            self.FtreImp[col].plot.bar(ax = ax, color = '#0073e6');
            ax.set_title(f"{col} Importances", **CFG.title_specs);
            ax.set(xlabel = '', ylabel = '');

            try:
                ax = axes[i,1];
            except:
                ax = axes[1];

            Clb.from_predictions(self.ytrain[0:len(self.OOF_Preds)], 
                                 self.OOF_Preds[col], 
                                 n_bins = 20, 
                                 ref_line = True,
                                 **{'color': '#0073e6', 'linewidth': 1.2, 
                                    'markersize': 3.75, 'marker': 'o', 'markerfacecolor': '#cc7a00'},
                                 ax = ax
                                );
            ax.set_title(f"{col} Calibration", **CFG.title_specs);
            ax.set(xlabel = '', ylabel = '',);
            ax.set_yticks(np.arange(0,1.01, 0.05), 
                          labels = np.round(np.arange(0,1.01, 0.05), 2), fontsize = 7.0);
            ax.set_xticks(np.arange(0,1.01, 0.05), 
                          labels = np.round(np.arange(0,1.01, 0.05), 2), 
                          fontsize = 6.25, 
                          rotation = 90
                         );
            ax.legend('');

        plt.tight_layout();
        plt.show();
                   
print();
collect();

In [ ]:
%%time

if CFG.ML == "Y":
    sel_cols = Xtrain.columns;
    PrintColor(f"\n---> Selected model columns");
    
    with np.printoptions(linewidth = 150):
        pprint(np.array(sel_cols));

print();
collect();

In [ ]:
%%time 

if CFG.ML == "Y":
    # Training the models with a CV analysis:-  
    md = MdlDeveloper(Xtrain, ytrain, Xtest, sel_cols = sel_cols, cat_cols = []);
    OOF_Preds, Mdl_Preds, Scores, TrainScores = \
    md.TrainMdl(test_preds_req = "Y", save_models = "Y");
    
    PrintColor(f"\n{'=' * 20} ML MODELS TRAINING AND CV {'=' * 20}\n", color = Fore.MAGENTA);
    md.DisplayScores();
    collect();
    
print();
collect();


In [ ]:
%%time 

if CFG.ML == "Y" and CFG.pseudo_lbl_req == "Y":
    Xtrain, ytrain = md.MakePseudoLbl(up_cutoff = CFG.pseudolbl_up, low_cutoff = CFG.pseudolbl_low);
    
    # Re-initializing the developer class with the new training set:-  
    md = MdlDeveloper(Xtrain, ytrain, Xtest, sel_cols = sel_cols, cat_cols = []);
    OOF_Preds, Mdl_Preds, Scores, TrainScores = md.TrainMdl(test_preds_req = "Y");
    
    PrintColor(f"\n{'=' * 20} ML MODELS TRAINING AND CV AFTER PSEUDO-LABELS {'=' * 20}\n", 
               color = Fore.MAGENTA
              );
    md.DisplayScores();
    collect();    
    
else:
    PrintColor(f"---> Pseudo Labels are not needed", Fore.RED);

print();
collect();

In [ ]:
%%time 

if CFG.ML == "Y":
    md.MakeMLPlots();
    
collect();
print();

In [ ]:
%%time 

# Saving the datasets:-
if CFG.ML == "Y":
    Mdl_Preds.index.name = "id";
    sub_fl = pp.sub_fl.copy();
    sub_fl[CFG.target] = 1 - Mdl_Preds["Ensemble"].values;
    
    # Blending with public work with post-processing:-
    sub1 = pd.read_csv(f"/kaggle/input/bank-churn-catboost-0-89626/submission.csv");
    
    sub_fl[CFG.target] = \
    np.select([sub1[CFG.target] == 0, sub1[CFG.target] == 1],
              [0,1],
              np.average(np.c_[sub_fl[CFG.target].values, sub1[CFG.target].values],
                         axis = 1, 
                         weights = [0.2, 0.8]
                        )
             );
    
    # Post-processing with public work:-    
    
    sub_fl.to_csv(f"Submission_V{CFG.version_nb}.csv", index = None);
    OOF_Preds.to_csv(f"OOF_Preds_V{CFG.version_nb}.csv");
    Mdl_Preds.to_csv(f"Mdl_Preds_V{CFG.version_nb}.csv");
    
    PrintColor(f"\nFinal submission file\n");
    display(sub_fl.head(10).style.format(precision = 3));
    
    PrintColor(f"\nSummary-submission post-processing\n");
    display(sub_fl.\
            assign(Check = np.select([sub_fl[CFG.target] == 0, sub_fl[CFG.target] == 1], [0,1], 2)
                  )["Check"].\
            value_counts(sort = True, ascending= True)
           );
    
collect();
print(); 